In [10]:
from mlflow.tracking import MlflowClient
import os
import mlflow
import boto3

In [11]:
os.environ["AWS_PROFILE"] = "park_pulse" 

TRACKING_SERVER_HOST = "ec2-54-90-62-154.compute-1.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

Interacting with the MLflow tracking server

The MlflowClient object allows us to interact with...

an MLflow Tracking Server that creates and manages experiments and runs.
an MLflow Registry Server that creates and manages registered models and model versions.
To instantiate it we need to pass a tracking URI and/or a registry URI

In [12]:
client = MlflowClient(tracking_uri=f"http://{TRACKING_SERVER_HOST}:5000")

client.search_experiments()

[<Experiment: artifact_location='s3://parkpulse-mlflow-artifacts/1', creation_time=1717408953570, experiment_id='1', last_update_time=1717408953570, lifecycle_stage='active', name='version 1', tags={}>,
 <Experiment: artifact_location='s3://parkpulse-mlflow-artifacts/0', creation_time=1717404637620, experiment_id='0', last_update_time=1717404637620, lifecycle_stage='active', name='Default', tags={}>]

In [13]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.r2 > 0.99",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.r2 DESC"]
)

In [14]:

for run in runs:
    print(f"run id: {run.info.run_id}, r2: {run.data.metrics['r2']:.4f}")

run id: 86b9fa1f26d9468f8ef60df4d6beaa3a, r2: 0.9994
run id: cc963ef7008f4640964e7a48b10d2c7d, r2: 0.9994
run id: cfb7bc36e6684efebbad54b39fd3ecff, r2: 0.9994
run id: 02e986f095c246f18e12e0cd97c3e0ea, r2: 0.9994
run id: 756f727c8f3d4d0e941404781629047b, r2: 0.9993


# Interacting with the Model Registry

***In this section We will use the MlflowClient instance to:***

Register a new version for the experiment nyc-taxi-regressor
Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.
Transition the version 4 to "Staging" and adding annotations to it.

In [18]:
best_run_id = runs[1].info.run_id
best_run_id

'cc963ef7008f4640964e7a48b10d2c7d'

In [19]:
model_uri = f"runs:/{best_run_id}/model"
mlflow.register_model(model_uri=model_uri, name="park-pulse-predictor")

Successfully registered model 'park-pulse-predictor'.
2024/06/03 12:54:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: park-pulse-predictor, version 1
Created version '1' of model 'park-pulse-predictor'.


<ModelVersion: aliases=[], creation_timestamp=1717412097500, current_stage='None', description='', last_updated_timestamp=1717412097500, name='park-pulse-predictor', run_id='cc963ef7008f4640964e7a48b10d2c7d', run_link='', source='s3://parkpulse-mlflow-artifacts/1/cc963ef7008f4640964e7a48b10d2c7d/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [20]:
model_name = "park-pulse-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

/var/folders/jx/_xm3vpjj6sbc2s0vkvwl7wp80000gp/T/ipykernel_12774/577744687.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


version: 1, stage: None
